## Import Libraries

In [1]:
import pandas as pd

## Import Dataframe

In [2]:
crime_log = pd.read_csv('../data/crime-log.csv')

## Clean the Data

In [3]:
crime_log.head()

,case_number,time_reported,time_occured,location_of_occurence,domestic_relationship,incident_type,criminal_offense,disposition
0,202300548,04/24/2023 04:56:00 pm,04/24/2023 04:56:00 pm-04/24/2023 04:56:00 pm,"907 HITT ST , COLUMBIA MO, 65201",NO,PROPERTY DAMAGE,PROPERTY DAMAGE,ACTIVE
1,202300546,04/24/2023 05:57:00 am,04/24/2023 05:57:00 am-04/24/2023 05:57:00 am,"3 HOSPITAL DR , COLUMBIA MO, 65201",NO,OFFICE INFORMATION (MUPD),OFFICE INFORMATION (MUPD),CLOSED
2,202300545,04/23/2023 06:26:00 pm,04/23/2023 06:26:00 pm-04/23/2023 06:26:00 pm,"904 ELM STREET/ EIGHTH STREET , COLUMBIA MO, 6...",NO,HARASSMENT,HARASSMENT,ACTIVE
3,202300544,04/23/2023 10:08:08 am,04/23/2023 08:08:41 am-04/23/2023 10:08:08 am,"1 HOSPITAL DR , COLUMBIA MO, 65201",NO,ASSAULT-SIMPLE,ASSAULT-SIMPLE,CLEARED BY ARREST
4,202300544,04/23/2023 10:08:08 am,04/23/2023 08:08:41 am-04/23/2023 10:08:08 am,"1 HOSPITAL DR , COLUMBIA MO, 65201",NO,OTHER OFFENSE - NOT REPORTABLE,ASSAULT-SIMPLE,CLEARED BY ARREST


- location_of_occurence: fix comma spacing
- time_occured: split into two separate columns
- convert time columns to datetime

In [4]:
# fix comma spacing for location
crime_log.location_of_occurence = crime_log.location_of_occurence.str.replace(' , ', ', ')

In [5]:
# split the time_occured column into two separate columns
crime_log[['time_started', 'time_ended']] = crime_log.time_occured.str.split('-', expand=True)
crime_log = crime_log.drop('time_occured', axis=1)

In [6]:
# convert time columns into datetime format
crime_log.time_reported = crime_log.time_reported.apply(lambda x: pd.to_datetime(x, format='%m/%d/%Y %I:%M:%S %p'))
crime_log.time_started = crime_log.time_started.apply(lambda x: pd.to_datetime(x, format='%m/%d/%Y %I:%M:%S %p'))
crime_log.time_ended = crime_log.time_ended.apply(lambda x: pd.to_datetime(x, format='%m/%d/%Y %I:%M:%S %p'))

In [7]:
# rearrange columns so that datetimes are adjacent
crime_log = crime_log.reindex(columns=['case_number', 
                                       'time_reported',
                                       'time_started', 
                                       'time_ended',
                                       'location_of_occurence',
                                       'domestic_relationship', 
                                       'incident_type', 
                                       'criminal_offense',
                                       'disposition'
                                      ])

In [8]:
# The forward slash followed by another street name in location_of_occurence was causing problems. Do minor cleanup here so that we can geocode with less error
crime_log.location_of_occurence = crime_log.location_of_occurence.str.replace(r'/[^,]+', '', regex=True)

# Some addresses have "UNIVERSITY HOSPITAL" in them, thus throwing errors when geocoding. It should actually be "HOSPITAL DRIVE". Use str.replace to clean.
crime_log.location_of_occurence = crime_log.location_of_occurence.str.replace('UNIVERSITY HOSPITAL', 'HOSPITAL DRIVE')

In [9]:
crime_log.head()

,case_number,time_reported,time_started,time_ended,location_of_occurence,domestic_relationship,incident_type,criminal_offense,disposition
0,202300548,2023-04-24 16:56:00,2023-04-24 16:56:00,2023-04-24 16:56:00,"907 HITT ST, COLUMBIA MO, 65201",NO,PROPERTY DAMAGE,PROPERTY DAMAGE,ACTIVE
1,202300546,2023-04-24 05:57:00,2023-04-24 05:57:00,2023-04-24 05:57:00,"3 HOSPITAL DR, COLUMBIA MO, 65201",NO,OFFICE INFORMATION (MUPD),OFFICE INFORMATION (MUPD),CLOSED
2,202300545,2023-04-23 18:26:00,2023-04-23 18:26:00,2023-04-23 18:26:00,"904 ELM STREET, COLUMBIA MO, 65211",NO,HARASSMENT,HARASSMENT,ACTIVE
3,202300544,2023-04-23 10:08:08,2023-04-23 08:08:41,2023-04-23 10:08:08,"1 HOSPITAL DR, COLUMBIA MO, 65201",NO,ASSAULT-SIMPLE,ASSAULT-SIMPLE,CLEARED BY ARREST
4,202300544,2023-04-23 10:08:08,2023-04-23 08:08:41,2023-04-23 10:08:08,"1 HOSPITAL DR, COLUMBIA MO, 65201",NO,OTHER OFFENSE - NOT REPORTABLE,ASSAULT-SIMPLE,CLEARED BY ARREST


## Export Data

In [10]:
crime_log.to_csv('../data/crime-log-clean.csv', index=False)